# Trait `AutoStructurable`

In [2]:
# import classes
from dataclasses import dataclass
from python_middlewareable import (
    RequestBase,
    MiddlewareBase,
    MiddlewareNextCallBase,
    MiddlewareableBase,
    DataStructurableRequestMixin,
    RequestDataBase,
    ResponseDataBase,
    TransportDataBase,
    AutoStructurable,
)


# define a payload structure
@dataclass
class RequestData(RequestDataBase):  # inherit from RequestDataBase
    name: str


# define a response structure
@dataclass
class ResponseData(ResponseDataBase):  # inherit from ResponseDataBase
    value: str = ""


# add the `DataStructurableRequestMixin` to the request class
@dataclass
class Request(
    DataStructurableRequestMixin[RequestData, ResponseData, TransportDataBase],
    RequestBase,
):
    pass


# define middlewares
class OneMiddleware(MiddlewareBase[Request]):
    async def handle(
        self, request: Request, next_call: MiddlewareNextCallBase[Request]
    ) -> Request:
        request.response_data.value = request.request_data.name

        request.response_data.value += " from OneMiddleware"

        result = await next_call(request)

        return result


class TwoMiddleware(MiddlewareBase[Request]):
    async def handle(
        self, request: Request, next_call: MiddlewareNextCallBase[Request]
    ) -> Request:
        request.response_data.value += " from TwoMiddleware"

        result = await next_call(request)

        return result


# add `AutoStructurable` to the middlewareable
class App(
    AutoStructurable[Request, ResponseData, TransportDataBase],
    MiddlewareableBase[Request],
):
    middlewares = [OneMiddleware, TwoMiddleware]

    # same classes passed to "AutoStructurable" above
    request_class = Request
    response_data_class = ResponseData
    transport_data_class = TransportDataBase


# middlewareable
app = App()

# process request
result = await app.process_middlewares_from_payload(
    data=RequestData(name="John")
)

# check the result
print(result)

# output:
# ResponseData(value='John from OneMiddleware from TwoMiddleware')

ResponseData(value='John from OneMiddleware from TwoMiddleware')
